![image.png](https://i.imgur.com/a3uAqnb.png)

### Exercise 1
Write a program (without using OOP) which takes in (x, y) ∈ +2 a tuple of
positive number representing the size of rectangular grid. It then takes an arbitrary number
of parameters (x, y), r, where the (x, y) represents the center of a circle and r represents
the radius. For each entry the program returns a "True" if the circle can be placed on the
grid without it intersecting with any other circles and then places the circle on the grid. Else
"False" if the circle can not be placed on the grid because it intersects with other previously
placed circle.

In [17]:
def can_place_circle(grid_size, circles, new_circle):
    x, y, r = new_circle
    width, height = grid_size

    # Check if circle fits inside the grid
    if not (r <= x <= width - r and r <= y <= height - r):
        return False

    # Check for intersection with existing circles
    for cx, cy, cr in circles:
        distance_sq = (x - cx) ** 2 + (y - cy) ** 2
        if distance_sq < (r + cr) ** 2:
            return False

    # Place circle
    circles.append(new_circle)
    return True

In [18]:
grid_size = (10, 10)
circles = []
print(can_place_circle(grid_size, circles, (2, 2, 1)))  # True
print(can_place_circle(grid_size, circles, (3, 2, 2)))  # False (intersects)
print(can_place_circle(grid_size, circles, (7, 7, 2)))  # True

True
False
True


### Exercise 2

Re-implement the above with OOP.

In [19]:
class CircleGrid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.circles = []

    def can_place(self, x, y, r):
        if not (r <= x <= self.width - r and r <= y <= self.height - r):
            return False
        for cx, cy, cr in self.circles:
            if (x - cx) ** 2 + (y - cy) ** 2 < (r + cr) ** 2:
                return False
        self.circles.append((x, y, r))
        return True

In [20]:
grid = CircleGrid(10, 10)
print(grid.can_place(2, 2, 1))  # True
print(grid.can_place(3, 2, 2))  # False
print(grid.can_place(7, 7, 2))  # True

True
False
True


### Exercise 3

Re-implement the above in "3" dimensional space. where "3" is the input of
the class constructor.

In [21]:
import math

class NDGrid:
    def __init__(self, dimensions):
        self.dimensions = dimensions  # tuple/list e.g. (10,10,10)
        self.spheres = []

    def can_place(self, center, radius):
        # Check if sphere fits in the grid
        for c, limit in zip(center, self.dimensions):
            if not (radius <= c <= limit - radius):
                return False

        # Check intersection with existing spheres
        for other_center, other_radius in self.spheres:
            dist_sq = sum((a - b) ** 2 for a, b in zip(center, other_center))
            if dist_sq < (radius + other_radius) ** 2:
                return False

        self.spheres.append((center, radius))
        return True



In [22]:
# Example usage in 3D
grid3d = NDGrid((10, 10, 10))
print(grid3d.can_place((2, 2, 2), 1))  # True
print(grid3d.can_place((2, 2, 3), 2))  # False (intersects)

True
False


### Exercise 4
Re-implement the above in 2 dimensional where we can now draw, circle,
squares, triangles, and rectangles. The format for the shapes is:
circle: x, y, r where x, y is center of circle and r is it’s radius
square: x, y, d where x, y is center of square and d is it’s side length
square: x, y, l, w where x, y is center of square and l, w are it’s length and width
triangle: x1, y1, x2, y2, x3, y3 where xi, yi represents the ith corner of the triangle


In [23]:
import math

class ShapeGrid2D:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.shapes = []

    def _distance(self, p1, p2):
        return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

    def can_place_circle(self, x, y, r):
        if not (r <= x <= self.width-r and r <= y <= self.height-r):
            return False
        for shape in self.shapes:
            if shape['type'] == 'circle':
                if self._distance((x,y), shape['center']) < r + shape['radius']:
                    return False
        self.shapes.append({'type':'circle', 'center':(x,y), 'radius':r})
        return True

    def can_place_square(self, x, y, d):
        half = d/2
        if not (half <= x <= self.width-half and half <= y <= self.height-half):
            return False
        self.shapes.append({'type':'square', 'center':(x,y), 'd':d})
        return True

    def can_place_rectangle(self, x, y, l, w):
        half_l, half_w = l/2, w/2
        if not (half_l <= x <= self.width-half_l and half_w <= y <= self.height-half_w):
            return False
        self.shapes.append({'type':'rectangle', 'center':(x,y), 'l':l, 'w':w})
        return True

    def can_place_triangle(self, points):
        # points = [(x1,y1),(x2,y2),(x3,y3)]
        if any(not (0 <= x <= self.width and 0 <= y <= self.height) for x,y in points):
            return False
        self.shapes.append({'type':'triangle', 'points':points})
        return True



In [24]:
grid2d = ShapeGrid2D(10, 10)

# These work
print(grid2d.can_place_circle(2, 2, 1))                 # True
print(grid2d.can_place_square(5, 5, 2))                 # True
print(grid2d.can_place_rectangle(8, 8, 3, 2))           # True
print(grid2d.can_place_triangle([(1,1),(2,1),(1,2)]))   # True

# These fail (go outside grid boundaries)
print(grid2d.can_place_circle(0, 0, 3))                 # False (circle goes outside grid)
print(grid2d.can_place_square(9, 9, 4))                 # False (square extends beyond grid)
print(grid2d.can_place_rectangle(9, 9, 4, 3))           # False (rectangle extends beyond grid)
print(grid2d.can_place_triangle([(0,0), (11,0), (5,5)]))# False (triangle corner outside grid)

True
True
True
True
False
False
False
False


### Extra
**Multiple Inheritance and Attribute Lookup:**

In Python, when an attribute (or method) is accessed through an object, the interpreter follows a well-defined order of lookup known as the Method Resolution Order (MRO). The lookup process follows these rules:


	1.	The attribute is first looked for in the class of the object itself.
	2.	If not found, it is then searched in the parent classes, from left to right as specified in the class definition.
	3.	If a parent itself has parents, the lookup continues recursively up the inheritance chain following the MRO.
	4.	If still not found, the lookup moves to the next parent class in order, and so on.
	5.	If no match is found anywhere in the hierarchy, Python raises an AttributeError.

Python provides useful built-in functions to work with classes and inheritance:

	•	isinstance(obj, Class) → checks whether an object is an instance of a given class (or its subclasses).
	•	issubclass(Class1, Class2) → checks whether Class1 is a subclass of Class2.

In [25]:
class ParentA:
    def __init__(self):
        self.attr_a = "Mother"

    def method_a(self):
        return "Message from the Mother"


class ParentB:
    def __init__(self):
        self.attr_b = "Father"

    def method_b(self):
        return "Message from the Father"

In [26]:
class Child(ParentA, ParentB):
    def __init__(self):
        super().__init__()         # Calls ParentA.__init__ because of MRO
        ParentB.__init__(self)     # Explicitly initialize ParentB
        self.attr_child = "Child"

    def method_child(self):
        return "Message from the Child"

In [27]:
c = Child()

# Access attributes from child and parents
print(c.attr_child)   # From Child
print(c.attr_a)       # From ParentA
print(c.attr_b)       # From ParentB

# Access methods
print(c.method_child())  # From Child
print(c.method_a())      # From ParentA
print(c.method_b())      # From ParentB

Child
Mother
Father
Message from the Child
Message from the Mother
Message from the Father


In [28]:
# Show Method Resolution Order
print(Child.__mro__)

(<class '__main__.Child'>, <class '__main__.ParentA'>, <class '__main__.ParentB'>, <class 'object'>)


In [30]:
# isinstance checks
print(isinstance(c, Child))     # True
print(isinstance(c, ParentA))   # True
print(isinstance(c, ParentB))   # True
print(isinstance(c, object))    # True

# issubclass checks
print(issubclass(Child, ParentA))   # True
print(issubclass(Child, ParentB))   # True
print(issubclass(Child, object))    # True
print(issubclass(ParentA, ParentB)) # False

True
True
True
True
True
True
True
False
